In [13]:
#importing
import pandas as pd
import json
import os
import saving_estimator
import glob

path = "/Users/Lauren/Desktop/UNSW/Interannual Variability/test/"
load_path = path + "input_load_pv_profiles/"
## should be csvs of half hourly load named {site_id}_profile.csv

## depends on how you want to assign tariffs but you want a dictionary that looks like this 
site_ids = [
    {"S0153": "Ausgrid"},
    {"S0189": "Energex"},
    {"S0209": "Ausgrid"},
    ##etc
    ]


In [14]:
### Lauren: I LEFT THE PV AND BATTERY STUFF IN THERE, JUST COMMENTED OUT. IN CASE YOU NEED IT LATER


## --------------------------------savings_calculator-----------------------------------------------
## A helper func to call savings_est_bill_calc row-by-row on a dataframe that has 
## the following columns:
## Customer_ID : a unique identifier for that customer data, can be float/int/str
## Tariff : string - the tariff ID to be used to calculate the bill. Requires that 
##          tariffs are stored in a dictionary with ID as the keys, tariff as vals
## 
## Also takes: 
##   tariff_dict - above mentioned dictionary w/ tariffs
##   load_and_gen_data - a dataframe with 3 columns: 'Datetime' (timestamps), 
##                       'kWh' (load) and 'PV' (solar profile)
##
## Returns: bill without solar, bill with solar, difference between the two (savings)
def savings_calculator(row, tariff_dict, load_and_gen_data):
    customer_id = row['Customer_ID']
    tariff_name = row['Tariff']
    tariff = tariff_dict[tariff_name]
    # system_size = row['Size']       # if system size isn't changing/relevant, delete this line
    
    customer_data = load_and_gen_data[load_and_gen_data['CUSTOMER_ID'] == customer_id]
    # customer_data['PV'] = customer_data['PV']
    # customer_data['kWh_PV'] = customer_data['kWh'] - (customer_data['PV'] * system_size)
    
    customer_data = customer_data.rename(columns={'Datetime' : 'TS'})
    customer_data['kWh']= customer_data['kWh']/2
    load_no_pv = customer_data[['TS', 'kWh']].copy()
    # load_with_pv = customer_data[['TS', 'kWh_PV']].copy()
    initial_bill = saving_estimator.bill_calculator(load_no_pv, tariff)

    # bill_with_solar = saving_estimator.bill_calculator(load_with_pv.rename(columns={'kWh_PV':'kWh'}), tariff)

    # # ----------- to do calculations with a battery ---------------------------- 

    # load_with_battery = saving_estimator.battery(tariff, customer_data[['TS', 'kWh', 'PV']].rename(columns={'kWh':'Load'}), 10, 10, 'Tesla')       # you can change the inputs here to change the size of Battery 
    # load_with_battery = load_with_battery[['TS', 'NetLoad']].rename(columns={'NetLoad':'kWh'})
    # bill_with_battery = saving_estimator.bill_calculator(load_with_battery, tariff)

    # savings = initial_bill['Retailer']['Bill'].values[0] - bill_with_solar['Retailer']['Bill'].values[0]
    # if savings < 0:
    #     print('No savings were made. Additional cost:'+ savings)
    #     savings = 0

    row['Initial Bill'] = initial_bill['Retailer']['Bill'].values[0]
    # row['Bill w/ Solar'] = bill_with_solar['Retailer']['Bill'].values[0]
    # row['Annual Savings'] = savings
    # row['Bill w/ Solar + Battery'] = bill_with_battery['Retailer']['Bill'].values[0]
    return row 
    


In [15]:
## for reading tariffs
#Read json files by passing filename only:
def read_json_file(filename):
    f = open(filename + ".json", 'r')
    data = json.loads(f.read())
    f.close()
    return data


In [ ]:
## call the function for each site
results = [] 

for site_dict in site_ids:
    for site, distributor in site_dict.items():
        for year in years:
            file_path = load_path + f'{site}_profile.csv'
            calculated_profile = pd.read_csv(file_path)
            
            calculated_profile.insert(0, 'CUSTOMER_ID', f'{site}')
            calculated_profile['TS'] = pd.to_datetime(calculated_profile['TS'])
            calculated_profile = calculated_profile[calculated_profile['TS'].dt.year == year]

            calculated_profile = pd.DataFrame(calculated_profile)
            new_order = ['CUSTOMER_ID','TS', 'kWh', 'PV']
            calculated_profile = calculated_profile[new_order]
            calculated_profile['kWh'] = calculated_profile['kWh']

            # Load tariffs
            tariffs_json = read_json_file(path +'AllTariffs_Retail_v9')## this is the tariffs json I used but you can change this
            tariffs = tariffs_json[0]['Tariffs']
            tariffs_dict = {
                tariff['Tariff ID']: tariff for tariff in tariffs
                if tariff.get('Distributor') == distributor
                if tariff.get('Type') == "Single_Rate" or tariff.get('Type') == "TOU" ##change this for whatever tariffs you need
            }

            tariff_ids = list(tariffs_dict.keys())

            customer_data = calculated_profile[calculated_profile['CUSTOMER_ID'] == site]
            customer_data.reset_index(drop=True, inplace=True)
            
            multi_index = pd.MultiIndex.from_product(
                [[site], tariff_ids],
                names=['Customer_ID', 'Tariff']
            )
            scenarios = pd.DataFrame(
                columns=['Initial Bill', 'State', 'Tariff Type', 'Year'],#'Bill w/ Solar', 'Annual Savings',
                index=multi_index
            )
            scenarios = scenarios.reset_index()
            scenarios[['Initial Bill']] = -1 #, 'Bill w/ Solar', 'Annual Savings', 'Bill w/ Solar + Battery'
            scenarios['State'] = scenarios['Tariff'].map(
                {id: tariffs_dict[id]['State'] for id in tariffs_dict}
            )
            scenarios['Tariff Type'] = scenarios['Tariff'].map(
                {id: tariffs_dict[id]['Type'] for id in tariffs_dict}
            )
            scenarios['Year'] = year
            scenarios = scenarios.apply(savings_calculator, axis=1, args=(tariffs_dict, customer_data))
            results.append(scenarios) 
         

# Concatenate the results into a single DataFrame
final_scenarios = pd.concat(results, ignore_index=True)

final_scenarios.to_csv(path+"bills.csv", index=False)
final_scenarios
